In [ ]:
from pptx import Presentation
from pptx.enum.shapes import MSO_SHAPE
from pptx.util import Pt
from pptx.dml.color import RGBColor

# Helper function to convert Western Arabic numerals to Persian numerals
def to_persian_number(number):
    persian_digits = {
        '0': '۰',
        '1': '۱',
        '2': '۲',
        '3': '۳',
        '4': '۴',
        '5': '۵',
        '6': '۶',
        '7': '۷',
        '8': '۸',
        '9': '۹'
    }
    return ''.join(persian_digits[digit] for digit in str(number))

def add_wrapping_progress_circles(pptx_path, output_path):
    prs = Presentation(pptx_path)
    num_slides = len(prs.slides)

    # Slide dimensions
    width = prs.slide_width
    height = prs.slide_height

    # Circle settings
    diameter = Pt(30)
    spacing = Pt(6)
    margin = Pt(10)
    step = diameter + spacing

    # Adjust max positions to avoid corner overlap
    max_vertical = int((height - 2 * margin - diameter) // step) + 1  # Account for corner space
    max_horizontal = int((width - 2 * margin - diameter) // step) + 1  # Account for corner space
    max_positions = 2 * max_vertical + 2 * max_horizontal - 4  # Subtract 4 to avoid corner overlap

    if num_slides > max_positions:
        raise ValueError("Too many slides to fit progress indicators on slide edges.")

    # Generate edge positions: right → bottom → left → top
    positions = []

    # Right edge (top to bottom, excluding bottom corner)
    for i in range(max_vertical):
        x = width - margin - diameter
        y = margin + i * step
        positions.append((x, y))

    # Bottom edge (right to left, excluding both corners)
    for i in range(1, max_horizontal - 1):
        x = width - margin - diameter - i * step
        y = height - margin - diameter
        positions.append((x, y))

    # Left edge (bottom to top, excluding top corner)
    for i in range(max_vertical - 1, -1, -1):
        x = margin
        y = margin + i * step
        positions.append((x, y))

    # Top edge (left to right, excluding both corners)
    for i in range(1, max_horizontal - 1):
        x = margin + i * step
        y = margin
        positions.append((x, y))

    # Add indicators to each slide
    for slide_index, slide in enumerate(prs.slides):
        for i in range(num_slides):
            x, y = positions[i]
            shape = slide.shapes.add_shape(
                MSO_SHAPE.OVAL, int(x), int(y), int(diameter), int(diameter)
            )
            fill = shape.fill
            fill.solid()
            shape.line.fill.background()

            if i < slide_index:
                fill.fore_color.rgb = RGBColor(255, 255, 0)  # Yellow for seen
            elif i == slide_index:
                fill.fore_color.rgb = RGBColor(255, 0, 0)    # Red for current
            else:
                fill.fore_color.rgb = RGBColor(200, 200, 200)  # Gray for not seen

            # Add number to:
            # - the current slide's circle (red one)
            # - the last circle (with total number of slides)
            if i == slide_index or i == num_slides - 1:
                text_frame = shape.text_frame
                if i == slide_index:
                    text_frame.text = to_persian_number(i + 1)  # Persian numerals
                elif i == num_slides - 1:
                    text_frame.text = to_persian_number(num_slides)  # Persian numerals

                p = text_frame.paragraphs[0]
                p.font.size = Pt(9)
                p.font.bold = True
                p.font.color.rgb = RGBColor(255, 255, 255)
                p.alignment = 1  # Center
                text_frame.word_wrap = False
                text_frame.auto_size = None
                text_frame.margin_top = 0
                text_frame.margin_bottom = 0
                text_frame.margin_left = 0
                text_frame.margin_right = 0

    last_slide = prs.slides[-1]
    textbox = last_slide.shapes.add_textbox(
        left=Pt(30), top=height - Pt(80), width=width - Pt(60), height=Pt(40)
    )
    tf = textbox.text_frame
    p = tf.paragraphs[0]
    p.text = f"Total Slides: {num_slides}"  # Unchanged, English style
    p.font.size = Pt(18)
    p.font.bold = True
    p.font.color.rgb = RGBColor(0, 0, 0)

    prs.save(output_path)
    print(f"Progress indicators and total count added to: {output_path}")

# Example usage
# add_wrapping_progress_circles("input.pptx", "output.pptx")
name = 'Title Lorem Ipsum'
add_wrapping_progress_circles(f'{name}.pptx', f'{name}_edit.pptx')

In [ ]:
from pptx import Presentation
from pptx.enum.shapes import MSO_SHAPE
from pptx.util import Pt
from pptx.dml.color import RGBColor

def add_wrapping_progress_circles(pptx_path, output_path):
    prs = Presentation(pptx_path)
    num_slides = len(prs.slides)

    # Slide dimensions
    width = prs.slide_width
    height = prs.slide_height

    # Circle settings
    diameter = Pt(30)  # slightly larger for text
    spacing = Pt(6)
    margin = Pt(10)
    step = diameter + spacing

    max_vertical = int((height - 2 * margin) // step)
    max_horizontal = int((width - 2 * margin) // step)
    max_positions = 2 * max_vertical + 2 * max_horizontal

    if num_slides > max_positions:
        raise ValueError("Too many slides to fit progress indicators on slide edges.")

    # Generate edge positions: right → bottom → left → top
    positions = []

    for i in range(max_vertical):
        x = width - margin - diameter
        y = margin + i * step
        positions.append((x, y))

    for i in range(max_horizontal):
        x = width - margin - diameter - i * step
        y = height - margin - diameter
        positions.append((x, y))

    for i in range(max_vertical):
        x = margin
        y = height - margin - diameter - i * step
        positions.append((x, y))

    for i in range(max_horizontal):
        x = margin + i * step
        y = margin
        positions.append((x, y))

    # Add indicators to each slide
    for slide_index, slide in enumerate(prs.slides):
        for i in range(num_slides):
            from pptx.enum.shapes import MSO_AUTO_SHAPE_TYPE

            # Hourglass position and size
            hg_left = Pt(20)
            hg_bottom = Pt(20)
            hg_height = Pt(100)
            hg_width = Pt(40)

            # Calculate top based on bottom
            hg_top = prs.slide_height - hg_bottom - hg_height

            # Add the hourglass outline (represented with a trapezoid to simulate a rough hourglass)
            hourglass = slide.shapes.add_shape(
                MSO_AUTO_SHAPE_TYPE.TRAPEZOID, int(hg_left), int(hg_top), int(hg_width), int(hg_height)
            )
            hourglass.rotation = 180  # Make it look like an hourglass
            hourglass.fill.solid()
            hourglass.fill.fore_color.rgb = RGBColor(200, 200, 200)
            hourglass.line.color.rgb = RGBColor(100, 100, 100)

            # Add the "sand" fill — simulate fill percentage
            progress_ratio = 1 - (slide_index / (num_slides - 1)) if num_slides > 1 else 1
            fill_height = int(hg_height * progress_ratio)
            fill_top = int(hg_top + (hg_height - fill_height))

            sand = slide.shapes.add_shape(
                MSO_SHAPE.RECTANGLE, int(hg_left), fill_top, int(hg_width), fill_height
            )
            sand.fill.solid()
            sand.fill.fore_color.rgb = RGBColor(255, 204, 0)  # Yellow sand
            sand.line.fill.background()


    last_slide = prs.slides[-1]
    textbox = last_slide.shapes.add_textbox(
        left=Pt(30), top=height - Pt(80), width=width - Pt(60), height=Pt(40)
    )
    tf = textbox.text_frame
    p = tf.paragraphs[0]
    p.text = f"Total Slides: {num_slides}"
    p.font.size = Pt(18)
    p.font.bold = True
    p.font.color.rgb = RGBColor(0, 0, 0)

    prs.save(output_path)
    print(f"Progress indicators and total count added to: {output_path}")


In [5]:
name = 'Title Lorem Ipsum'
add_wrapping_progress_circles(f'{name}.pptx', f'{name}_hourglass.pptx')

Progress indicators and total count added to: Title Lorem Ipsum_hourglass.pptx


In [64]:
from pptx import Presentation
from pptx.enum.shapes import MSO_SHAPE
from pptx.util import Pt
from pptx.dml.color import RGBColor

def add_wrapping_progress_circles(pptx_path, output_path):
    prs = Presentation(pptx_path)
    num_slides = len(prs.slides)

    # Slide dimensions
    width = prs.slide_width
    height = prs.slide_height

    # Circle settings
    diameter = Pt(30)  # slightly larger for text
    spacing = Pt(6)
    margin = Pt(10)
    step = diameter + spacing

    max_vertical = int((height - 2 * margin) // step)
    max_horizontal = int((width - 2 * margin) // step)
    max_positions = 2 * max_vertical + 2 * max_horizontal

    if num_slides > max_positions:
        raise ValueError("Too many slides to fit progress indicators on slide edges.")

    # Generate edge positions: right → bottom → left → top
    positions = []

    for i in range(max_vertical):
        x = width - margin - diameter
        y = margin + i * step
        positions.append((x, y))

    for i in range(max_horizontal):
        x = width - margin - diameter - i * step
        y = height - margin - diameter
        positions.append((x, y))

    for i in range(max_vertical):
        x = margin
        y = height - margin - diameter - i * step
        positions.append((x, y))

    for i in range(max_horizontal):
        x = margin + i * step
        y = margin
        positions.append((x, y))

    # Add indicators to each slide
    for slide_index, slide in enumerate(prs.slides):
        for i in range(num_slides):
            from pptx.enum.shapes import MSO_SHAPE

            # Hourglass dimensions
            hg_left = Pt(20)
            hg_bottom = Pt(20)
            hg_height = Pt(100)
            hg_width = Pt(40)

            # Base position
            hg_top = prs.slide_height - hg_bottom - hg_height
            center_x = int(hg_left + hg_width // 2)
            middle_y = int(hg_top + hg_height // 2)

            # Sand progress
            progress = slide_index / (num_slides - 1) if num_slides > 1 else 1

            # Top sand (shrinks)
            top_height = int((1 - progress) * (hg_height // 2))
            top_shape = slide.shapes.add_shape(
                MSO_SHAPE.ISOSCELES_TRIANGLE,
                int(hg_left), middle_y - top_height,
                int(hg_width), top_height
            )
            top_shape.rotation = 180  # Flip to point downward
            top_shape.fill.solid()
            top_shape.fill.fore_color.rgb = RGBColor(255, 204, 0)  # Yellow
            top_shape.line.fill.background()

            # Bottom sand (grows)
            bottom_height = int(progress * (hg_height // 2))
            bottom_shape = slide.shapes.add_shape(
                MSO_SHAPE.ISOSCELES_TRIANGLE,
                int(hg_left), middle_y,
                int(hg_width), bottom_height
            )
            bottom_shape.fill.solid()
            bottom_shape.fill.fore_color.rgb = RGBColor(255, 204, 0)  # Yellow
            bottom_shape.line.fill.background()

            # Optional: outline hourglass with transparent triangle frames
            outline = slide.shapes.add_shape(
                MSO_SHAPE.TRAPEZOID,
                int(hg_left), int(hg_top),
                int(hg_width), int(hg_height)
            )
            outline.rotation = 180
            outline.fill.background()
            outline.line.color.rgb = RGBColor(100, 100, 100)



    last_slide = prs.slides[-1]
    textbox = last_slide.shapes.add_textbox(
        left=Pt(30), top=height - Pt(80), width=width - Pt(60), height=Pt(40)
    )
    tf = textbox.text_frame
    p = tf.paragraphs[0]
    p.text = f"Total Slides: {num_slides}"
    p.font.size = Pt(18)
    p.font.bold = True
    p.font.color.rgb = RGBColor(0, 0, 0)

    prs.save(output_path)
    print(f"Progress indicators and total count added to: {output_path}")


In [65]:
name = 'Title Lorem Ipsum'
add_wrapping_progress_circles(f'{name}.pptx', f'{name}_hourglass2.pptx')

Progress indicators and total count added to: Title Lorem Ipsum_hourglass2.pptx


In [89]:
from pptx import Presentation
from pptx.enum.shapes import MSO_SHAPE
from pptx.util import Pt
from pptx.dml.color import RGBColor

def add_wrapping_progress_circles(pptx_path, output_path):
    prs = Presentation(pptx_path)
    num_slides = len(prs.slides)

    # Slide dimensions
    width = prs.slide_width
    height = prs.slide_height

    # Circle settings
    diameter = Pt(30)  # slightly larger for text
    spacing = Pt(6)
    margin = Pt(10)
    step = diameter + spacing

    max_vertical = int((height - 2 * margin) // step)
    max_horizontal = int((width - 2 * margin) // step)
    max_positions = 2 * max_vertical + 2 * max_horizontal

    if num_slides > max_positions:
        raise ValueError("Too many slides to fit progress indicators on slide edges.")

    # Generate edge positions: right → bottom → left → top
    positions = []

    for i in range(max_vertical):
        x = width - margin - diameter
        y = margin + i * step
        positions.append((x, y))

    for i in range(max_horizontal):
        x = width - margin - diameter - i * step
        y = height - margin - diameter
        positions.append((x, y))

    for i in range(max_vertical):
        x = margin
        y = height - margin - diameter - i * step
        positions.append((x, y))

    for i in range(max_horizontal):
        x = margin + i * step
        y = margin
        positions.append((x, y))

    # Add indicators to each slide
    for slide_index, slide in enumerate(prs.slides):
        for i in range(num_slides):
            from pptx.enum.shapes import MSO_SHAPE

            # Position & size
            margin_left = Pt(20)
            margin_bottom = Pt(20)
            hourglass_height = Pt(100)
            hourglass_width = Pt(40)

            top = prs.slide_height - margin_bottom - hourglass_height
            center_y = top + hourglass_height // 2
            center_x = margin_left + hourglass_width // 2

            # Sand progress ratio
            progress = slide_index / (num_slides - 1) if num_slides > 1 else 1

            # Colors
            sand_color = RGBColor(255, 204, 0)
            glass_color = RGBColor(180, 180, 180)

            # -- Draw top sand (shrinking ellipse) --
            top_sand_height = int((1 - progress) * (hourglass_height // 2))
            top_sand = slide.shapes.add_shape(
                MSO_SHAPE.OVAL,
                int(margin_left),
                int(center_y - top_sand_height),
                int(hourglass_width),
                int(top_sand_height)
            )
            top_sand.fill.solid()
            top_sand.fill.fore_color.rgb = sand_color
            top_sand.fill.fore_color.transparency = 0.1
            top_sand.line.fill.background()

            # --- Bottom bowl (glass outline) ---
            bottom_bowl_top = int(center_y)
            bottom_bowl_height = int(hourglass_height // 2)

            bottom_ellipse = slide.shapes.add_shape(
                MSO_SHAPE.OVAL,
                int(margin_left),
                bottom_bowl_top,
                int(hourglass_width),
                bottom_bowl_height
            )
            bottom_ellipse.fill.background()
            bottom_ellipse.line.color.rgb = glass_color
            bottom_ellipse.line.width = Pt(1.25)

            # --- Bottom fill (fake sand, ellipse growing from bottom) ---
            # Simulate with an oval whose height increases and is vertically offset
            max_fill_height = bottom_bowl_height
            current_fill_height = int(progress * max_fill_height)

            # To center horizontally and place vertically aligned with bowl bottom
            fill_top = bottom_bowl_top + max_fill_height - current_fill_height

            sand_fill = slide.shapes.add_shape(
                MSO_SHAPE.OVAL,
                int(margin_left),
                fill_top,
                int(hourglass_width),
                current_fill_height
            )
            sand_fill.fill.solid()
            sand_fill.fill.fore_color.rgb = sand_color
            sand_fill.fill.fore_color.transparency = 0.1
            sand_fill.line.fill.background()



            # -- Draw neck as a narrow rounded rectangle --
            neck = slide.shapes.add_shape(
                MSO_SHAPE.ROUNDED_RECTANGLE,
                int(margin_left + hourglass_width // 3),
                int(center_y - Pt(2)),
                int(hourglass_width // 3),
                int(Pt(4))
            )
            neck.fill.solid()
            neck.fill.fore_color.rgb = sand_color
            neck.fill.fore_color.transparency = 0.3
            neck.line.fill.background()

            # -- Draw glass outline (optional) --
            glass = slide.shapes.add_shape(
                MSO_SHAPE.ROUNDED_RECTANGLE,
                int(margin_left),
                int(top),
                int(hourglass_width),
                int(hourglass_height)
            )
            glass.fill.background()
            glass.line.color.rgb = glass_color
            glass.line.width = Pt(1.5)




    last_slide = prs.slides[-1]
    textbox = last_slide.shapes.add_textbox(
        left=Pt(30), top=height - Pt(80), width=width - Pt(60), height=Pt(40)
    )
    tf = textbox.text_frame
    p = tf.paragraphs[0]
    p.text = f"Total Slides: {num_slides}"
    p.font.size = Pt(18)
    p.font.bold = True
    p.font.color.rgb = RGBColor(0, 0, 0)

    prs.save(output_path)
    print(f"Progress indicators and total count added to: {output_path}")

In [90]:
name = 'Title Lorem Ipsum'
add_wrapping_progress_circles(f'{name}.pptx', f'{name}_hourglass3.pptx')

Progress indicators and total count added to: Title Lorem Ipsum_hourglass3.pptx


In [92]:
from pptx import Presentation
from pptx.enum.shapes import MSO_SHAPE
from pptx.util import Pt
from pptx.dml.color import RGBColor

def add_wrapping_progress_circles(pptx_path, output_path):
    prs = Presentation(pptx_path)
    num_slides = len(prs.slides)

    # Slide dimensions
    width = prs.slide_width
    height = prs.slide_height

    # Circle settings
    diameter = Pt(30)  # slightly larger for text
    spacing = Pt(6)
    margin = Pt(10)
    step = diameter + spacing

    max_vertical = int((height - 2 * margin) // step)
    max_horizontal = int((width - 2 * margin) // step)
    max_positions = 2 * max_vertical + 2 * max_horizontal

    if num_slides > max_positions:
        raise ValueError("Too many slides to fit progress indicators on slide edges.")

    # Generate edge positions: right → bottom → left → top
    positions = []

    for i in range(max_vertical):
        x = width - margin - diameter
        y = margin + i * step
        positions.append((x, y))

    for i in range(max_horizontal):
        x = width - margin - diameter - i * step
        y = height - margin - diameter
        positions.append((x, y))

    for i in range(max_vertical):
        x = margin
        y = height - margin - diameter - i * step
        positions.append((x, y))

    for i in range(max_horizontal):
        x = margin + i * step
        y = margin
        positions.append((x, y))

    # Add indicators to each slide
    for slide_index, slide in enumerate(prs.slides):
        for i in range(num_slides):
            from pptx.enum.shapes import MSO_SHAPE

            # Position & size
            margin_left = Pt(20)
            margin_bottom = Pt(20)
            hourglass_height = Pt(100)
            hourglass_width = Pt(40)

            top = prs.slide_height - margin_bottom - hourglass_height
            center_y = top + hourglass_height // 2
            center_x = margin_left + hourglass_width // 2

            # Sand progress ratio
            progress = slide_index / (num_slides - 1) if num_slides > 1 else 1

            # Colors
            sand_color = RGBColor(255, 204, 0)
            glass_color = RGBColor(180, 180, 180)

            # -- Draw top sand (shrinking ellipse) --
            top_sand_height = int((1 - progress) * (hourglass_height // 2))
            top_sand = slide.shapes.add_shape(
                MSO_SHAPE.OVAL,
                int(margin_left),
                int(center_y - top_sand_height),
                int(hourglass_width),
                int(top_sand_height)
            )
            top_sand.fill.solid()
            top_sand.fill.fore_color.rgb = sand_color
            top_sand.fill.fore_color.transparency = 0.1
            top_sand.line.fill.background()

            from pptx.enum.shapes import MSO_AUTO_SHAPE_TYPE

            # --- Bottom bowl (outline) ---
            bottom_bowl_top = int(center_y)
            bottom_bowl_height = int(hourglass_height // 2)

            bottom_ellipse = slide.shapes.add_shape(
                MSO_SHAPE.OVAL,
                int(margin_left),
                bottom_bowl_top,
                int(hourglass_width),
                bottom_bowl_height
            )
            bottom_ellipse.fill.background()
            bottom_ellipse.line.color.rgb = glass_color
            bottom_ellipse.line.width = Pt(1.25)

            # --- Bottom sand fill (rounded rectangle to simulate piling sand) ---
            max_fill_height = bottom_bowl_height - 2  # Padding to stay inside bowl
            current_fill_height = max(1, int(progress * max_fill_height))  # Ensure at least 1px height

            fill_top = bottom_bowl_top + (max_fill_height - current_fill_height) + 1  # Keep at bottom

            sand_fill = slide.shapes.add_shape(
                MSO_AUTO_SHAPE_TYPE.ROUNDED_RECTANGLE,
                int(margin_left + 1),
                fill_top,
                int(hourglass_width - 2),
                current_fill_height
            )
            sand_fill.fill.solid()
            sand_fill.fill.fore_color.rgb = sand_color
            sand_fill.fill.fore_color.transparency = 0.1
            sand_fill.line.fill.background()


            # --- Bottom fill (fake sand, ellipse growing from bottom) ---
            # Simulate with an oval whose height increases and is vertically offset
            max_fill_height = bottom_bowl_height
            current_fill_height = int(progress * max_fill_height)

            # To center horizontally and place vertically aligned with bowl bottom
            fill_top = bottom_bowl_top + max_fill_height - current_fill_height

            sand_fill = slide.shapes.add_shape(
                MSO_SHAPE.OVAL,
                int(margin_left),
                fill_top,
                int(hourglass_width),
                current_fill_height
            )
            sand_fill.fill.solid()
            sand_fill.fill.fore_color.rgb = sand_color
            sand_fill.fill.fore_color.transparency = 0.1
            sand_fill.line.fill.background()



            # -- Draw neck as a narrow rounded rectangle --
            neck = slide.shapes.add_shape(
                MSO_SHAPE.ROUNDED_RECTANGLE,
                int(margin_left + hourglass_width // 3),
                int(center_y - Pt(2)),
                int(hourglass_width // 3),
                int(Pt(4))
            )
            neck.fill.solid()
            neck.fill.fore_color.rgb = sand_color
            neck.fill.fore_color.transparency = 0.3
            neck.line.fill.background()

            # -- Draw glass outline (optional) --
            glass = slide.shapes.add_shape(
                MSO_SHAPE.ROUNDED_RECTANGLE,
                int(margin_left),
                int(top),
                int(hourglass_width),
                int(hourglass_height)
            )
            glass.fill.background()
            glass.line.color.rgb = glass_color
            glass.line.width = Pt(1.5)




    last_slide = prs.slides[-1]
    textbox = last_slide.shapes.add_textbox(
        left=Pt(30), top=height - Pt(80), width=width - Pt(60), height=Pt(40)
    )
    tf = textbox.text_frame
    p = tf.paragraphs[0]
    p.text = f"Total Slides: {num_slides}"
    p.font.size = Pt(18)
    p.font.bold = True
    p.font.color.rgb = RGBColor(0, 0, 0)

    prs.save(output_path)
    print(f"Progress indicators and total count added to: {output_path}")

In [94]:
name = 'Title Lorem Ipsum'
add_wrapping_progress_circles(f'{name}.pptx', f'{name}_hourglass4.pptx')

Progress indicators and total count added to: Title Lorem Ipsum_hourglass4.pptx


In [99]:
import math
from pptx import Presentation
from pptx.util import Pt
from pptx.dml.color import RGBColor

def add_hourglass_progress(pptx_path, output_path):
    prs = Presentation(pptx_path)
    num_slides = len(prs.slides)

    # Slide dimensions
    width = prs.slide_width
    height = prs.slide_height

    # Hourglass settings
    hg_diameter = Pt(40)
    hg_radius = hg_diameter / 2
    hg_x = Pt(20)
    hg_y = height - hg_diameter - Pt(20)
    steps = num_slides
    slice_height = hg_diameter / (2 * steps)

    def add_circle_slice(slide, x_center, y_center, radius, y_start, y_end, color):
        for y in range(int(y_start), int(y_end)):
            rel_y = y - y_center
            if abs(rel_y) > radius:
                continue
            chord_half_width = math.sqrt(radius**2 - rel_y**2)
            x0 = x_center - chord_half_width
            width = 2 * chord_half_width
            shape = slide.shapes.add_shape(
                autoshape_type_id=1,  # Rectangle
                left=int(x0),
                top=y,
                width=int(width),
                height=1
            )
            shape.fill.solid()
            shape.fill.fore_color.rgb = color
            shape.line.fill.background()

    for i, slide in enumerate(prs.slides):
        # Calculate the amount of sand to add to the bottom part of the hourglass
        slices_to_add = int((i + 1) * (hg_radius * 2) / num_slides)  # Sand filled

        # Top part is empty (falling)
        slices_to_remove = int(i * (hg_radius * 2) / num_slides)  # Emptying top part

        top_start = hg_y
        top_end = hg_y + hg_radius
        top_slice_start = top_start
        top_slice_end = top_start + slices_to_remove * slice_height
        add_circle_slice(slide, hg_x + hg_radius, hg_y + hg_radius, hg_radius, top_slice_start, top_slice_end, RGBColor(255, 255, 255))

        # Bottom filled part
        bottom_start = hg_y + hg_radius
        bottom_end = hg_y + 2 * hg_radius
        bottom_slice_start = bottom_end - slices_to_add * slice_height
        bottom_slice_end = bottom_end
        add_circle_slice(slide, hg_x + hg_radius, hg_y + hg_radius, hg_radius, bottom_slice_start, bottom_slice_end, RGBColor(255, 192, 0))  # Orange sand

    prs.save(output_path)
    return output_path

In [101]:
name = 'Title Lorem Ipsum'
add_wrapping_progress_circles(f'{name}.pptx', f'{name}_hourglass5.pptx')

Progress indicators and total count added to: Title Lorem Ipsum_hourglass5.pptx
